In [14]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy import sparse
import time
import sys
sys.path.append('tools/')


from sklearn.model_selection import KFold


In [3]:
trainFile = 'round1_ijcai_18_train_20180301.txt'
testFile = 'round1_ijcai_18_test_a_20180301.txt'
dataDir = '../../Data/advertisment/Raw/'

In [4]:
dfTrain = pd.read_table(dataDir+trainFile,sep=' ')
dfTest = pd.read_table(dataDir+testFile,sep=' ')

In [5]:
print(dfTrain.dtypes)
print(dfTrain.shape)

instance_id                    int64
item_id                        int64
item_category_list            object
item_property_list            object
item_brand_id                  int64
item_city_id                   int64
item_price_level               int64
item_sales_level               int64
item_collected_level           int64
item_pv_level                  int64
user_id                        int64
user_gender_id                 int64
user_age_level                 int64
user_occupation_id             int64
user_star_level                int64
context_id                     int64
context_timestamp              int64
context_page_id                int64
predict_category_property     object
shop_id                        int64
shop_review_num_level          int64
shop_review_positive_rate    float64
shop_star_level                int64
shop_score_service           float64
shop_score_delivery          float64
shop_score_description       float64
is_trade                       int64
d

In [6]:
featInput = []
keyList = ['instance_id','item_id','user_id','context_id','shop_id','is_trade']
for var in dfTrain.columns:
    if not var in keyList and dfTrain[var].dtypes != 'object':
        featInput.append(var)
print(featInput)
print(len(featInput))

['item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level', 'context_timestamp', 'context_page_id', 'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description']
18


<font color=#0099ff size=5 face="黑体">处理日期</font>

In [7]:
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return value.tm_hour


In [8]:
dfTrain['context_timestamp_hour'] = dfTrain['context_timestamp'].map(timestamp_datetime)
try: featInput.remove('context_timestamp')
except: print('context_timestamp has been deleted')
if not 'context_timestamp_hour' in featInput:
    featInput.append('context_timestamp_hour')

<font color=#0099ff size=5 face="黑体">One Hot变形</font>

In [9]:
dummyInput = []
removeInput = []
for var in featInput:
    if 'int' in str(dfTrain[var].dtypes):
        print(var)
        dfTrain[var].replace({-1:np.nan},inplace=True)
        dfTmp = pd.get_dummies(dfTrain[var],var)
        dfTrain = pd.concat([dfTrain,dfTmp],axis=1)
        try:
            removeInput.append(var)
            dummyInput +=dfTmp.columns.tolist()
        dfTrain = pd.concat([dfTrain,dfTmp],axis=1)
        except:
            print('Removed')
for var in removeInput:
    featInput.remove(var)
featInput+=dummyInput

item_brand_id
item_city_id
item_price_level
item_sales_level
item_collected_level
item_pv_level
user_gender_id
user_age_level
user_occupation_id
user_star_level
context_page_id
shop_review_num_level
shop_star_level
context_timestamp_hour


In [10]:
print(dfTrain.columns)
print(dfTrain.shape)

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level',
       ...
       'context_timestamp_hour_14', 'context_timestamp_hour_15',
       'context_timestamp_hour_16', 'context_timestamp_hour_17',
       'context_timestamp_hour_18', 'context_timestamp_hour_19',
       'context_timestamp_hour_20', 'context_timestamp_hour_21',
       'context_timestamp_hour_22', 'context_timestamp_hour_23'],
      dtype='object', length=2397)
(478138, 2397)


<font color=#0099ff size=5 face="黑体">CV 创建模型输入</font>

In [11]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
for trainIndex,testIndex in cv.split(dfTrain[keyList]):
    trainX,testX = dfTrain.loc[trainIndex,featInput],dfTrain.loc[testIndex,featInput]
    trainY,testY = dfTrain.loc[trainIndex,'is_trade'],dfTrain.loc[testIndex,'is_trade']

In [20]:
fm =fm_tool(learningRate=0.01,epochs=10)
fm.fit(TrainX=testX,Trainy=testY,k=32,verbose=True)

C:\Users\Leo Mao\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in subtract


epoch--0 in 2333.95 s


MemoryError: 

In [19]:
class fm_tool(object):
    def __init__(self,epochs=5,learningRate=0.01):
        self.epochs = epochs
        self.learningRate = learningRate

        
    def data_type(self,data):
        if not type(data) is sp.sparse.csr.csr_matrix:
            return sparse.csr_matrix(data)
        else:
            return data
        
        
    def predict_raw(self,TrainX):
        trainX = self.data_type(data=TrainX)
        result = (trainX*self.weight_first) + self.weight_constant + 0.5*(np.power(trainX*self.weight_second.T,2)-trainX.power(2)*np.power(self.weight_second,2).T).sum(axis=1)
        return result
    
    def predict(self,TrainX):
        trainX = self.data_type(data=TrainX)
        return (1/(1+np.exp(-1*self.predict_raw(trainX))))
        
    def fit(self,TrainX,Trainy,k,ValX=None,Valy=None,verbose=False):
        ###init the model
        self.k = k
        trainX = self.data_type(data=TrainX)
        #trainyTmp = trainy.reshape(trainy.shape[0],1)
        trainyTmp = np.matrix(Trainy).T
        baseMatrix = np.matrix([1]*trainX.shape[1])
        self.weight_constant = 0
        self.weight_first = np.matrix(np.random.randn(trainX.shape[1])).T
        self.weight_second = np.matrix(np.random.randn(self.k,trainX.shape[1]))
        
        for n in range(self.epochs):
            start_time = time.time()
            gradientBase = -1*trainyTmp + self.predict(trainX)
            self.learningRate = 0.9*self.learningRate
            for i in range(trainX.shape[0]):

                recordTmp = trainX[i].T
                gradientBaseTmp = gradientBase[i]
                          
                
                gradientInteraction = self.weight_second*recordTmp
                
                self.weight_constant = self.weight_constant - self.learningRate*gradientBaseTmp
                self.weight_first = self.weight_first - recordTmp*self.learningRate*gradientBaseTmp
                self.weight_second = self.weight_second - self.learningRate*gradientBaseTmp[0,0]*(recordTmp*gradientInteraction.T-recordTmp.power(2)*baseMatrix*self.weight_second.T).T
            if verbose:
                print('epoch--%d in %.2f s'%(n,time.time()-start_time))
                print('Train Set accuracy: %.4f'%self.accuracy(trainX,Trainy))
                try: print('Validation Set accuracy: %.4f'%self.accuracy(valX,valy))
                except: continue
            
    def accuracy(self,TrainX,Trainy):
        trainX = self.data_type(data=TrainX)
        trainyTmp = np.matrix(Trainy).T
        return -1*(np.log(1- trainyTmp-self.predict(trainX)+2*(trainyTmp*self.predict(trainX).T).diagonal().T).mean())

In [15]:
class fm_tool(object):
    def __init__(self,epochs=5,learningRate=0.01,k=32):
        self.epochs = epochs
        self.learningRate = learningRate
        self.k = k
        
        
    def predict_raw(self,trainX):
        result = (trainX.T*self.weight_first).sum(axis=0).reshape(trainX.shape[0],1) + self.weight_constant
        '''for i in range(0,trainX.shape[1]-1):
            for j in range(i+1,trainX.shape[1]):
                result = result + np.dot(self.weight_second[i,:],self.weight_second[j,:])*(trainX[:,i]*trainX[:,j]).reshape(trainX.shape[0],1)'''
        return result
    
    def predict(self,trainX):
        return (1/(1+np.exp(-1*self.predict_raw(trainX))))
        
    def fit(self,trainX,trainy,k,valX=None,valy=None,verbose=False):
        ###init the model
        self.k = k
        trainyTmp = trainy.reshape(trainy.shape[0],1)
        self.weight_constant = 0
        self.weight_first = np.random.randn(trainX.shape[1],1)
        self.weight_second = np.random.randn(trainX.shape[1],self.k)
        
        '''if batchSize <= 0:
            batchSize = trainXTmp.shape[0]'''
        for n in range(self.epochs):
            start_time = time.time()
            #gradientBase = -1*trainyTmp*np.exp(-1*trainyTmp*self.predict_raw(trainXTmp))/(1+np.exp(-1*trainyTmp*self.predict_raw(trainXTmp)))
            print(1)
            gradientBase = -1*trainyTmp + self.predict(trainX)
            #print(gradientBase)
            print(2)
            self.learningRate = 0.9*self.learningRate
            for i in range(trainX.shape[0]):
                print(3)
                recordTmp = trainX[i].reshape(trainX.shape[1],1)
                print(4)
                gradientBaseTmp = gradientBase[i].reshape(gradientBase.shape[1],1)
                gradientInteraction = (self.weight_second*recordTmp).sum(axis=0)
                self.weight_constant = self.weight_constant - self.learningRate*gradientBaseTmp
                self.weight_first = self.weight_first - self.learningRate*gradientBaseTmp*recordTmp   
                self.weight_second = self.weight_second - self.learningRate*gradientBaseTmp*(np.outer(recordTmp,gradientInteraction)-np.power(recordTmp,2)*self.weight_second)
            if verbose:
                print('epoch--%d in %.2f s'%(n,time.time()-start_time))
                print('Train Set accuracy: %.4f'%self.accuracy(trainX,trainy))
                try: print('Validation Set accuracy: %.4f'%self.accuracy(valX,valy))
                except: continue
            
    def accuracy(self,trainX,trainy):
        return -1*(np.log(1- trainy.reshape(trainy.shape[0],1)-self.predict(trainX)+2*trainy.reshape(trainy.shape[0],1)*self.predict(trainX)).mean())
            

In [11]:
trainX,testX = dfTrain.loc[0:300000,featInput].values,dfTrain.loc[300000:,featInput].values
trainY,testY = dfTrain.loc[0:300000,'is_trade'].values,dfTrain.loc[300000:,'is_trade'].values

In [12]:
del trainX

In [46]:
type(tt)

numpy.ndarray

In [16]:
fm =fm_tool(learningRate=0.01,epochs=100)
fm.fit(trainX=testX,trainy=testY,k=32,verbose=True)

1


KeyboardInterrupt: 

In [27]:
start_time = time.time()

test = StestX*Stest_weight

print(time.time()-start_time)

0.0690000057220459


In [28]:
test.shape

(300001, 1)

In [26]:
from scipy import sparse
StestX = sparse.csr_matrix(testX)
Stest_weight = sparse.csr_matrix(test_weight)

In [29]:
type(StestX)

scipy.sparse.csr.csr_matrix

In [22]:
testX = np.matrix(testX)
test_weight = np.matrix(fm.weight_first)

In [20]:
start_time = time.time()

test = np.dot(testX,fm.weight_first)

print(time.time()-start_time)

77.65606927871704


In [21]:
test.shape

(300001, 1)

In [24]:
tt = trainX.values

In [25]:
tt.shape

(382511, 2373)